In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Ovarian_Cancer"

# Input paths
tcga_root_dir = "../../input/TCGA"

# Output paths
out_data_file = "../../output/preprocess/Ovarian_Cancer/TCGA.csv"
out_gene_data_file = "../../output/preprocess/Ovarian_Cancer/gene_data/TCGA.csv"
out_clinical_data_file = "../../output/preprocess/Ovarian_Cancer/clinical_data/TCGA.csv"
json_path = "../../output/preprocess/Ovarian_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# Step 1: Find the directory corresponding to Ovarian Cancer
import os

# List all directories in TCGA root directory
tcga_dirs = os.listdir(tcga_root_dir)

# Find the directory that matches our trait: Ovarian_Cancer
ovarian_dirs = [dir_name for dir_name in tcga_dirs if "ovarian" in dir_name.lower()]

if not ovarian_dirs:
    print(f"No matching directory found for trait: {trait}")
    # Record that this trait is not available and exit
    validate_and_save_cohort_info(
        is_final=False,
        cohort="TCGA",
        info_path=json_path,
        is_gene_available=False,
        is_trait_available=False
    )
else:
    # Select the most relevant directory
    selected_dir = ovarian_dirs[0]  # Should be 'TCGA_Ovarian_Cancer_(OV)'
    cohort_dir = os.path.join(tcga_root_dir, selected_dir)
    
    # Step 2: Get file paths for clinical and genetic data
    clinical_file_path, genetic_file_path = tcga_get_relevant_filepaths(cohort_dir)
    
    # Step 3: Load the files
    clinical_df = pd.read_csv(clinical_file_path, sep='\t', index_col=0)
    genetic_df = pd.read_csv(genetic_file_path, sep='\t', index_col=0)
    
    # Step 4: Print column names of clinical data
    print("Clinical data columns:")
    print(clinical_df.columns.tolist())


Clinical data columns:
['_INTEGRATION', '_PANCAN_CNA_PANCAN_K8', '_PANCAN_Cluster_Cluster_PANCAN', '_PANCAN_DNAMethyl_PANCAN', '_PANCAN_RPPA_PANCAN_K8', '_PANCAN_UNC_RNAseq_PANCAN_K16', '_PANCAN_miRNA_PANCAN', '_PANCAN_mirna_OV', '_PANCAN_mutation_PANCAN', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'age_at_initial_pathologic_diagnosis', 'anatomic_neoplasm_subdivision', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'clinical_stage', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathologic_diagnosis', 'days_to_last_followup', 'days_to_new_tumor_event_additional_surgery_procedure', 'days_to_new_tumor_event_after_initial_treatment', 'eastern_cancer_oncology_group', 'followup_case_report_form_submission_reason', 'followup_treatment_success', 'form_completion_date', 'gender', 'histological_type', 'history_of_neoadjuvant_treatment', 'icd_10', 'icd_o_3_histolo

### Step 2: Find Candidate Demographic Features

In [3]:
# Identifying columns related to age
candidate_age_cols = ['age_at_initial_pathologic_diagnosis', 'days_to_birth']

# Identifying columns related to gender
candidate_gender_cols = ['gender']

# The directory structure might require navigating to a specific cohort directory
# Let's first check if we can identify the ovarian cancer (OV) cohort directory
import os

# Look for the OV cohort directory within the TCGA root directory
possible_cohort_dirs = [os.path.join(tcga_root_dir, d) for d in os.listdir(tcga_root_dir) if 'OV' in d]

if possible_cohort_dirs:
    cohort_dir = possible_cohort_dirs[0]
    try:
        clinical_file_path, _ = tcga_get_relevant_filepaths(cohort_dir)
        clinical_df = pd.read_csv(clinical_file_path, index_col=0, sep='\t')
        
        # Preview age-related columns
        age_preview = {}
        for col in candidate_age_cols:
            if col in clinical_df.columns:
                age_preview[col] = clinical_df[col].head(5).tolist()
        
        # Preview gender-related columns
        gender_preview = {}
        for col in candidate_gender_cols:
            if col in clinical_df.columns:
                gender_preview[col] = clinical_df[col].head(5).tolist()
        
        print("Age columns preview:")
        print(age_preview)
        print("\nGender columns preview:")
        print(gender_preview)
    except (IndexError, FileNotFoundError) as e:
        print(f"Could not access clinical data files: {e}")
        print("Identified candidate columns without preview:")
        print(f"Age columns: {candidate_age_cols}")
        print(f"Gender columns: {candidate_gender_cols}")
else:
    print("Could not locate Ovarian Cancer cohort directory.")
    print("Identified candidate columns without preview:")
    print(f"Age columns: {candidate_age_cols}")
    print(f"Gender columns: {candidate_gender_cols}")


Age columns preview:
{'age_at_initial_pathologic_diagnosis': [nan, nan, nan, nan, nan], 'days_to_birth': [nan, nan, nan, nan, nan]}

Gender columns preview:
{'gender': [nan, nan, nan, nan, nan]}


### Step 3: Select Demographic Features

In [4]:
# Examine age columns
print("Age columns inspection:")
age_col = None
for column in ['age_at_initial_pathologic_diagnosis', 'days_to_birth']:
    if column in clinical_df.columns:
        non_null_values = clinical_df[column].notna().sum()
        total_values = len(clinical_df[column])
        non_null_percentage = (non_null_values / total_values) * 100 if total_values > 0 else 0
        print(f"Column '{column}': {non_null_values}/{total_values} non-null values ({non_null_percentage:.2f}%)")
        
        # Check a few values
        sample_values = clinical_df[column].dropna().head(5).tolist()
        print(f"Sample values: {sample_values}")
        
        # Choose column with higher non-null percentage
        if non_null_percentage > 50:  # Reasonable threshold for usability
            if age_col is None or non_null_percentage > (clinical_df[age_col].notna().sum() / len(clinical_df[age_col])) * 100:
                age_col = column

# Examine gender column
print("\nGender column inspection:")
gender_col = None
if 'gender' in clinical_df.columns:
    non_null_values = clinical_df['gender'].notna().sum()
    total_values = len(clinical_df['gender'])
    non_null_percentage = (non_null_values / total_values) * 100 if total_values > 0 else 0
    print(f"Column 'gender': {non_null_values}/{total_values} non-null values ({non_null_percentage:.2f}%)")
    
    # Check a few values
    sample_values = clinical_df['gender'].dropna().head(5).tolist()
    print(f"Sample values: {sample_values}")
    
    # Choose gender column if it has reasonable non-null percentage
    if non_null_percentage > 50:  # Reasonable threshold for usability
        gender_col = 'gender'

# Output chosen columns
print("\nChosen demographic columns:")
print(f"age_col = {age_col}")
print(f"gender_col = {gender_col}")


Age columns inspection:
Column 'age_at_initial_pathologic_diagnosis': 607/630 non-null values (96.35%)
Sample values: [78.0, 70.0, 60.0, 55.0, 78.0]
Column 'days_to_birth': 596/630 non-null values (94.60%)
Sample values: [-28848.0, -25786.0, -21963.0, -20271.0, -28626.0]

Gender column inspection:
Column 'gender': 607/630 non-null values (96.35%)
Sample values: ['FEMALE', 'FEMALE', 'FEMALE', 'FEMALE', 'FEMALE']

Chosen demographic columns:
age_col = age_at_initial_pathologic_diagnosis
gender_col = gender


### Step 4: Feature Engineering and Validation

In [5]:
# Step 1: Extract and standardize clinical features
# Selecting the trait, age, and gender from clinical data
selected_clinical_df = tcga_select_clinical_features(
    clinical_df, 
    trait=trait, 
    age_col=age_col, 
    gender_col=gender_col
)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")
print(f"Clinical data shape: {selected_clinical_df.shape}")

# Step 2: Normalize gene symbols in gene expression data
# First, inspect the structure of the genetic data
print(f"Original genetic data shape: {genetic_df.shape}")
print(f"Sample column names (first 5): {list(genetic_df.columns[:5])}")
print(f"Sample row indices (first 5): {list(genetic_df.index[:5])}")

# The genetic data likely has genes as rows and samples as columns already
# No need to transpose, just normalize gene symbols
normalized_gene_df = normalize_gene_symbols_in_index(genetic_df)
print(f"Normalized gene data shape: {normalized_gene_df.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_df.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# Step 3: Link clinical and genetic data
# Debug: Print sample indices to understand format
print("Clinical data index examples:", selected_clinical_df.index[:5].tolist())
print("Gene data column examples:", normalized_gene_df.columns[:5].tolist())

# Extract sample IDs from both clinical and genetic data and standardize format
clinical_sample_ids = selected_clinical_df.index
gene_sample_ids = normalized_gene_df.columns

# Find common samples using exact matching
common_samples = list(set(clinical_sample_ids).intersection(set(gene_sample_ids)))
print(f"Found {len(common_samples)} common samples between clinical and gene data")

# If no exact matches, try to find pattern-based matches
if not common_samples:
    # Extract TCGA barcodes up to the sample portion (first 12 characters) from clinical IDs
    clinical_barcodes = [sample_id[:12] for sample_id in clinical_sample_ids if sample_id.startswith('TCGA-')]
    # Check if the gene data columns contain these barcodes
    for col in gene_sample_ids:
        for barcode in clinical_barcodes:
            if barcode in col:
                print(f"Found potential match: {barcode} in {col}")
    
    # Alternative approach: prepare subsets for visual comparison
    print("Sample clinical IDs (first 10):", clinical_sample_ids[:10].tolist())
    print("Sample gene expression column names (first 10):", list(gene_sample_ids[:10]))

# Create linked data by combining clinical and gene expression data
if common_samples:
    # Filter clinical data to include only common samples
    clinical_subset = selected_clinical_df.loc[common_samples]
    # Extract gene expression data for common samples
    gene_subset = normalized_gene_df[common_samples]
    # Transpose gene subset to have samples as rows
    gene_subset_t = gene_subset.transpose()
    # Combine the dataframes
    linked_data = pd.concat([clinical_subset, gene_subset_t], axis=1)
else:
    # If there are no common samples, create an empty DataFrame
    linked_data = pd.DataFrame()
    print("No common samples found between clinical and gene expression data.")

print(f"Linked data shape: {linked_data.shape}")
print(f"Number of samples: {linked_data.shape[0]}")
print(f"Number of features (including clinical): {linked_data.shape[1]}")

# Step 4: Handle missing values systematically
if not linked_data.empty:
    cleaned_data = handle_missing_values(linked_data, trait_col=trait)
    print(f"Cleaned data shape after handling missing values: {cleaned_data.shape}")
    
    # Step 5: Determine if trait and demographic features are severely biased
    is_trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)
    
    # Preview results
    print("\nPreview of cleaned linked data:")
    print(cleaned_data.head(3))
else:
    cleaned_data = pd.DataFrame()
    is_trait_biased = True  # If we can't link data, consider it biased/unusable
    print("Cannot proceed with empty linked data")

# Step 6: Conduct final quality validation
# Validate and save cohort information
is_gene_available = normalized_gene_df.shape[0] > 0
is_trait_available = selected_clinical_df.shape[0] > 0
note = "Ovarian cancer dataset with gene expression data from TCGA. "

if linked_data.empty:
    note += "Failed to link clinical and genetic data due to no common sample IDs. This may be due to different sample ID formats."
    is_trait_biased = True  # Consider it biased if we can't link data
elif is_trait_biased:
    note += "The trait distribution is severely biased, making the dataset unsuitable for analysis."
else:
    note += "The dataset appears to be balanced and suitable for analysis."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort="TCGA",
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_trait_biased,
    df=cleaned_data,
    note=note
)

# Step 7: Save linked data if usable
if is_usable and not cleaned_data.empty:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Dataset is usable. Saved linked data to {out_data_file}")
else:
    print("Dataset is not usable due to severe bias or data quality issues. Linked data not saved.")

Saved clinical data to ../../output/preprocess/Ovarian_Cancer/clinical_data/TCGA.csv
Clinical data shape: (630, 3)
Original genetic data shape: (20530, 308)
Sample column names (first 5): ['TCGA-61-1910-01', 'TCGA-61-1728-01', 'TCGA-09-1666-01', 'TCGA-24-1469-01', 'TCGA-04-1348-01']
Sample row indices (first 5): ['ARHGEF10L', 'HIF3A', 'RNF17', 'RNF10', 'RNF11']
Normalized gene data shape: (19848, 308)


Saved normalized gene data to ../../output/preprocess/Ovarian_Cancer/gene_data/TCGA.csv
Clinical data index examples: ['TCGA-01-0628-11', 'TCGA-01-0629-11', 'TCGA-01-0630-11', 'TCGA-01-0631-11', 'TCGA-01-0633-11']
Gene data column examples: ['TCGA-61-1910-01', 'TCGA-61-1728-01', 'TCGA-09-1666-01', 'TCGA-24-1469-01', 'TCGA-04-1348-01']
Found 308 common samples between clinical and gene data
Linked data shape: (308, 19851)
Number of samples: 308
Number of features (including clinical): 19851


Cleaned data shape after handling missing values: (308, 19851)
Quartiles for 'Ovarian_Cancer':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Ovarian_Cancer' in this dataset is severely biased.

Quartiles for 'Age':
  25%: 51.0
  50% (Median): 58.0
  75%: 67.0
Min: 30.0
Max: 87.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '0.0' with 308 occurrences. This represents 100.00% of the dataset.
The distribution of the feature 'Gender' in this dataset is severely biased.




Preview of cleaned linked data:
                 Ovarian_Cancer   Age      A1BG  A1BG-AS1      A1CF       A2M  \
TCGA-13-0899-01               1  60.0 -0.897074 -0.900583 -1.742862 -0.245477   
TCGA-13-0920-01               1  65.0 -1.064674 -3.359783 -1.742862 -1.960177   
TCGA-25-1314-01               1  42.0  1.276526  1.628517 -1.742862 -1.307177   

                    A2ML1    A4GALT   A4GNT      AAA1  ...    ZWILCH  \
TCGA-13-0899-01  3.562506  0.181199 -1.1892 -0.359541  ... -0.175754   
TCGA-13-0920-01 -2.981794 -2.446901 -1.1892 -0.359541  ...  0.748346   
TCGA-25-1314-01 -1.851794 -2.102501 -1.1892 -0.359541  ...  0.080646   

                    ZWINT      ZXDA      ZXDB      ZXDC    ZYG11A   ZYG11B  \
TCGA-13-0899-01 -0.127832 -0.366196 -0.543364 -0.346067 -1.341914 -1.23273   
TCGA-13-0920-01  2.069668 -0.242796 -0.217864  0.245633  0.964886  0.46607   
TCGA-25-1314-01  0.435468  0.471604 -0.141664 -0.006667  5.056986  0.37887   

                      ZYX     ZZEF1     